# Toronto Clustering

### Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

### Import the table from Wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

data = pd.read_html(url, header = 0)

data = data[0]

### Ignore cells with a borough that is Not assigned.

In [3]:
data = data[data['Borough'] != 'Not assigned']

In [4]:
data.columns = ['Postcode', 'Borough', 'Neighborhood']

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
data[data['Neighborhood'] == 'Not assigned'] # This code returned only cell index 9

,Postcode,Borough,Neighborhood
9,M9A,Queen's Park,Not assigned


In [6]:
data.loc[9, 'Neighborhood'] = data.loc[9, 'Borough'] # Update the cell index 9
data.loc[9, 'Neighborhood']

"Queen's Park"

### Combine rows with the same Postcode

In [7]:
data = (data.groupby(['Postcode', 'Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index())

In [8]:
data.shape

(103, 3)

### Getting latitude and longitude for each row from file

In [9]:
coordinatesDf = pd.read_csv('Geospatial_Coordinates.csv')
coordinatesDf.tail(1)

,Postal Code,Latitude,Longitude
102,M9W,43.706748,-79.594054


In [10]:
Latitude = coordinatesDf['Latitude']
Longitude = coordinatesDf['Longitude']

In [11]:
data['Latitude'] = Latitude
data['Longitude'] = Longitude

In [12]:
data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Explore the neighborhoods in Toronto

#### Import dependencies

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [14]:
address = "Toronto, On"

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
mapToronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

# Add markers to each region 
for lat, long, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = neighborhood + " " + borough
    label = folium.Popup(label, parse_html = True)
    
    folium.CircleMarker([lat, long], radius = 5, popup = label, color = 'blue', 
                        fill = True, fill_color='#3186cc', fill_opacity=0.7, 
                        parse_html=False).add_to(mapToronto)
    
# This for goes through each row in the dataframe, extract the variables into the new for variables, create a label 
# with the retrived variables neighborhood and borough from df and creates a new folium marker (circle) for each 
# row with theirs lat and long and add the markers to the map previously created.
    
mapToronto

# Using Foursquare API

In [16]:
CLIENT_ID = ''
CLIENT_SECRET = '' 
VERSION = '20180605' # Foursquare API version

In [17]:
lat_test = data.loc[0, 'Latitude']
long_test = data.loc[0, 'Longitude']
name_test = data.loc[0, 'Borough']
print("Region: {}, latitude = {}, longitude = {}." .format(name_test, lat_test, long_test))

Region: Scarborough, latitude = 43.806686299999996, longitude = -79.19435340000001.


### Using API to check TOP 20 venues in the region (area of 1km)

In [18]:
limit = 20
radius = 1000

# url with Venues/explore 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat_test,
    long_test,
    radius,
    limit)
print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=5DYP1FFNGEI5QB4CDZGREB41CSYFISWGIQQGGJY1QPPZREEE&client_secret=JVHJPEDZPV40K4DBW24IEFQKPSAKYXKHG00TRQXEDAY4PONI&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=20


In [19]:
results = requests.get(url).json()

### Functions to clean the results

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']

from pandas.io.json import json_normalize
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy's,Fast Food Restaurant,43.807448,-79.199056
3,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
4,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
5,Tim Hortons,Coffee Shop,43.802000,-79.198169
6,Harvey's,Fast Food Restaurant,43.800106,-79.198258
7,Lee Valley,Hobby Shop,43.803161,-79.199681
8,Tim Hortons / Esso,Coffee Shop,43.801863,-79.199296
9,Bella's Lechon,Filipino Restaurant,43.801291,-79.198378


### Getting the top 10 Venues for each neighborhood and creating a dataframe toronto_venues

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )


In [23]:
toronto_venues.shape
# Now we have approximately the top 10-20 venues of each Neighborhood
# Total of 1805 top venues in Toronto

(1814, 7)

# Examin each neighborhood venues 

In [24]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [25]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
toronto_onehot = toronto_onehot.set_index('Neighborhood').reset_index()

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Track,Trail,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# Filtering only columns of interest

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_onehot = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Track,Trail,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.1,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0

In [27]:
toronto_onehot = toronto_onehot[['Neighborhood', 'Breakfast Spot', 'Café', 'Coffee Shop','Fast Food Restaurant']]

In [28]:
toronto_onehot

,Neighborhood,Breakfast Spot,Café,Coffee Shop,Fast Food Restaurant
0,"Adelaide, King, Richmond",0.000000,0.050000,0.050000,0.000000
1,Agincourt,0.050000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.050000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.000000,0.055556
4,"Alderwood, Long Branch",0.000000,0.000000,0.050000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.100000,0.050000
6,Bayview Village,0.000000,0.071429,0.000000,0.071429
7,"Bedford Park, Lawrence Manor East",0.000000,0.050000,0.150000,0.050000
8,Berczy Park,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.076923,0.000000,0.076923


In [29]:
toronto_venues = toronto_onehot
toronto_venues['Total'] = toronto_venues.sum(axis = 1)

In [30]:
toronto_venues.sort_values(by=['Total'], ascending = False)

,Neighborhood,Breakfast Spot,Café,Coffee Shop,Fast Food Restaurant,Total
44,"First Canadian Place, Underground city",0.000000,0.200000,0.150000,0.000000,0.350000
35,"Dovercourt Village, Dufferin",0.000000,0.200000,0.150000,0.000000,0.350000
32,"Design Exchange, Toronto Dominion Centre",0.000000,0.150000,0.200000,0.000000,0.350000
27,"Commerce Court, Victoria Hotel",0.000000,0.250000,0.100000,0.000000,0.350000
63,Lawrence Park,0.000000,0.222222,0.111111,0.000000,0.333333
70,Northwest,0.000000,0.000000,0.333333,0.000000,0.333333
80,Scarborough Village,0.000000,0.000000,0.083333,0.250000,0.333333
97,Woburn,0.000000,0.000000,0.200000,0.100000,0.300000
77,"Rouge, Malvern",0.000000,0.000000,0.117647,0.176471,0.294118
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.133333,0.133333,0.000000,0.266667


In [31]:
toronto_venues.shape

(101, 6)

In [32]:
data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
new_df = pd.merge(data, toronto_venues, on = 'Neighborhood')

In [34]:
new_df.sort_values(by = ['Total'], ascending = False)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Breakfast Spot,Café,Coffee Shop,Fast Food Restaurant,Total
75,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,0.000000,0.200000,0.150000,0.000000,0.350000
69,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.000000,0.200000,0.150000,0.000000,0.350000
60,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0.000000,0.250000,0.100000,0.000000,0.350000
59,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0.000000,0.150000,0.200000,0.000000,0.350000
101,M9W,Etobicoke,Northwest,43.706748,-79.594054,0.000000,0.000000,0.333333,0.000000,0.333333
43,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.000000,0.222222,0.111111,0.000000,0.333333
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.000000,0.000000,0.083333,0.250000,0.333333
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.000000,0.000000,0.200000,0.100000,0.300000
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.000000,0.000000,0.117647,0.176471,0.294118
67,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0.000000,0.133333,0.133333,0.000000,0.266667


In [35]:
mapToronto = folium.Map(location = [43.653963, -79.387207], zoom_start = 11)
for neighborhood, lat, long, total in zip(new_df['Neighborhood'], new_df['Latitude'], new_df['Longitude'], new_df['Total']):
    label = neighborhood + ' score: ' + str(round(total, 3))
    label = folium.Popup(label, parse_html = True)
    
    folium.CircleMarker([lat, long], radius = 5, popup = label, color = 'red', fill=True, fill_opacity=1).add_to(mapToronto)

mapToronto

# We have the score of popularity for each neighborhood

# Let's set our areas of interest, where popularity of this type of shop is greater than 0.2

In [36]:
restricted_df = new_df[new_df['Total'] > 0.2]
restricted_df.sort_values(by = 'Total', ascending = False)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Breakfast Spot,Café,Coffee Shop,Fast Food Restaurant,Total
60,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0.0000,0.250000,0.100000,0.000000,0.350000
75,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,0.0000,0.200000,0.150000,0.000000,0.350000
69,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.0000,0.200000,0.150000,0.000000,0.350000
59,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0.0000,0.150000,0.200000,0.000000,0.350000
43,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.0000,0.222222,0.111111,0.000000,0.333333
101,M9W,Etobicoke,Northwest,43.706748,-79.594054,0.0000,0.000000,0.333333,0.000000,0.333333
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0000,0.000000,0.083333,0.250000,0.333333
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0000,0.000000,0.200000,0.100000,0.300000
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0000,0.000000,0.117647,0.176471,0.294118
67,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0.0000,0.133333,0.133333,0.000000,0.266667


In [37]:
restricted_df.shape

(22, 10)

## Now we have 22 potential good neighborhoods for coffee/breakfast related venues

In [42]:
restrictedMap = folium.Map(location = [43.653963, -79.387207], zoom_start = 11)

for lat, long in zip(restricted_df['Latitude'], restricted_df['Longitude']):
     folium.CircleMarker([lat, long], radius = 5, color = 'red', 
                         fill=True, fill_opacity=1).add_to(restrictedMap)
restrictedMap

In [60]:
cluster_df = new_df[['Total']]

In [62]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 3, init = 'k-means++')

In [63]:
labels = kmeans.fit_predict(cluster_df)

In [68]:
new_df['Cluster'] = labels

In [69]:
new_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Breakfast Spot,Café,Coffee Shop,Fast Food Restaurant,Total,Cluster
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,0.0,0.117647,0.176471,0.294118,1
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.2,0.0,0.000000,0.000000,0.200000,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,0.0,0.050000,0.100000,0.150000,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,0.0,0.200000,0.100000,0.300000,1
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,0.0,0.100000,0.000000,0.100000,2


# Ploting cluster 0

In [81]:
cluster_0 = new_df[new_df['Cluster'] == 0]

clusteredMap = folium.Map(location = [43.653963, -79.387207], zoom_start = 11)
for lat, long in zip(cluster_0['Latitude'], cluster_0['Longitude']):
    folium.CircleMarker([lat, long], radius = 5, color = 'blue', fill = True, fill_opacity = 1).add_to(clusteredMap)

In [82]:
cluster_1 = new_df[new_df['Cluster'] == 1]
for lat, long in zip(cluster_1['Latitude'], cluster_1['Longitude']):
    folium.CircleMarker([lat, long], radius = 5, color = 'green', fill = True, fill_opacity = 1).add_to(clusteredMap)

In [84]:
cluster_2 = new_df[new_df['Cluster'] == 2]
for lat, long in zip(cluster_2['Latitude'], cluster_2['Longitude']):
    folium.CircleMarker([lat, long], radius = 5, color = 'red', fill = True, fill_opacity = 1).add_to(clusteredMap)

In [85]:
clusteredMap